In [4]:
import os
import data_processing_tool as dpt
from datetime import timedelta, date, datetime
# from args_parameter import args
from PrepareData import ACCESS_BARRA_v4
from torch.utils.data import Dataset,random_split
import torch,os,torchvision
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim

# from PIL import Image

import time
from sklearn.model_selection import StratifiedShuffleSplit
# import model
# from model import my_model
# import utility
from tqdm import tqdm
import math
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr,compare_mse
import platform
from torch.autograd import Variable
import argparse
def set_args():
    parser = argparse.ArgumentParser(description='BARRA_R and ACCESS-S!')
    parser.add_argument('--args_test', type=int, default=0,
                            help='testing parameters input')
    parser.add_argument('--debug', action='store_true',
                        help='Enables debug mode')
    parser.add_argument('--template', default='.',
                        help='You can set various templates in option.py')

    # Hardware specifications
    parser.add_argument('--n_threads', type=int, default=0,
                        help='number of threads for data loading')
    parser.add_argument('--cpu', action='store_true',
                        help='use cpu only')
    parser.add_argument('--n_GPUs', type=int, default=1,
                        help='number of GPUs')
    parser.add_argument('--seed', type=int, default=1,
                        help='random seed')

    # Data specifications
    parser.add_argument('--pr', type=bool, 
                    default=True,
                    help='add-on pr?')

    parser.add_argument('--dem', type=bool, 
                    default=True,
                    help='add-on dem?') 
    parser.add_argument('--psl', type=bool, 
                    default=False,
                    help='add-on psl?') 
    parser.add_argument('--zg', type=bool, 
                    default=False,
                    help='add-on zg?') 
    parser.add_argument('--tasmax', type=bool, 
                    default=False,
                    help='add-on tasmax?') 
    parser.add_argument('--tasmin', type=bool, 
                    default=False,
                    help='add-on tasmin?')

    parser.add_argument('--leading_time_we_use', type=int, 
                    default=7,
                    help='add-on tasmin?')




    parser.add_argument('--ensemble', type=int, 
                    default=2,
                    help='total ensambles is 11') 


    parser.add_argument('--channels', type=float, 
                        default=0,
                        help='channel of data_input must') 
    #[111.85, 155.875, -44.35, -9.975]
    parser.add_argument('--domain', type=list, 
                        default=[112.9, 154.25, -43.7425, -9.0],
                        help='dataset directory')    


    parser.add_argument('--file_ACCESS_dir', type=str, 
                        default="F:/climate/access-s1/pr/daily/",

                        help='dataset directory')
    parser.add_argument('--file_BARRA_dir', type=str, 
                        default="C:/Users/JIA059/barra/",
                        help='dataset directory')

    parser.add_argument('--file_DEM_dir', type=str, 
                        default="../DEM/",
                        help='dataset directory')

#     parser.add_argument('--nine2nine', type=bool, 
#                         default=True,
#                         help='whether rainfall acculate from 9am to 9am')
#     parser.add_argument('--date_minus_one', type=int, 
#                         default=1,
#                         help='whether rainfall acculate from yesterday(1)/today(0) 9am to tody/tomorrow 9am')


    parser.add_argument('--dir_demo', type=str, default='../test',
                        help='demo image directory')
    #     parser.add_argument('--data_train', type=str, default='BARRA_R',
    #                         help='train dataset name')
    #     parser.add_argument('--data_test', type=str, default='DIV2K',
    #                         help='test dataset name')
    parser.add_argument('--benchmark_noise', action='store_true',
                        help='use noisy benchmark sets')
    parser.add_argument('--n_train', type=int, default=800,
                        help='number of training set')
    parser.add_argument('--n_val', type=int, default=10,
                        help='number of validation set')
    parser.add_argument('--offset_val', type=int, default=800,
                        help='validation index offest')
    parser.add_argument('--ext', type=str, default='sep',
                        help='dataset file extension')
    parser.add_argument('--scale', default='4',
                        help='super resolution scale')
    parser.add_argument('--patch_size', type=int, default=96,
                        help='output patch size')
    #??????????????????????????????????????????????????
    parser.add_argument('--rgb_range', type=int, default=400,
                        help='maximum value of RGB')
    parser.add_argument('--n_colors', type=int, default=1,
                        help='number of color channels to use')
    parser.add_argument('--noise', type=str, default='.',
                        help='Gaussian noise std.')
    parser.add_argument('--chop', action='store_true',
                        help='enable memory-efficient forward')

    # Model specifications
    parser.add_argument('--model', default='RCAN',
                        help='model name')

    parser.add_argument('--act', type=str, default='relu',
                        help='activation function')
    parser.add_argument('--pre_train', type=str, default='.',
                        help='pre-trained model directory')
    parser.add_argument('--extend', type=str, default='.',
                        help='pre-trained model directory')
    parser.add_argument('--n_resblocks', type=int, default=16,
                        help='number of residual blocks')
    parser.add_argument('--n_feats', type=int, default=64,
                        help='number of feature maps')
    parser.add_argument('--res_scale', type=float, default=1,
                        help='residual scaling')
    parser.add_argument('--shift_mean', default=True,
                        help='subtract pixel mean from the input')
    parser.add_argument('--precision', type=str, default='single',
                        choices=('single', 'half'),
                        help='FP precision for test (single | half)')

    # Training specifications

    parser.add_argument('--train_name', type=str, default='temp01',
                        help='the trainning name of the set')
    parser.add_argument('--reset', action='store_true',
                        help='reset the training')
    parser.add_argument('--test_every', type=int, default=1000,
                        help='do test per every N batches')
    parser.add_argument('--epochs', type=int, default=300,
                        help='number of epochs to train')
    parser.add_argument('--batch_size', type=int, default=32,
                        help='input batch size for training')
    parser.add_argument('--split_batch', type=int, default=1,
                        help='split the batch into smaller chunks')
    parser.add_argument('--self_ensemble', action='store_true',
                        help='use self-ensemble method for test')
    parser.add_argument('--test_only', action='store_true',
                        help='set this option to test the model')
    parser.add_argument('--gan_k', type=int, default=1,
                        help='k value for adversarial loss')

    # Optimization specifications
    parser.add_argument('--lr', type=float, default=1e-4,
                        help='learning rate')
    parser.add_argument('--lr_decay', type=int, default=200,
                        help='learning rate decay per N epochs')
    parser.add_argument('--decay_type', type=str, default='step',
                        help='learning rate decay type')
    parser.add_argument('--gamma', type=float, default=0.5,
                        help='learning rate decay factor for step decay')
    parser.add_argument('--optimizer', default='ADAM',
                        choices=('SGD', 'ADAM', 'RMSprop'),
                        help='optimizer to use (SGD | ADAM | RMSprop)')
    parser.add_argument('--momentum', type=float, default=0.9,
                        help='SGD momentum')
    parser.add_argument('--beta1', type=float, default=0.9,
                        help='ADAM beta1')
    parser.add_argument('--beta2', type=float, default=0.999,
                        help='ADAM beta2')
    parser.add_argument('--epsilon', type=float, default=1e-8,
                        help='ADAM epsilon for numerical stability')
    parser.add_argument('--weight_decay', type=float, default=0,
                        help='weight decay')

    # Loss specifications
    parser.add_argument('--loss', type=str, default='1*L1',
                        help='loss function configuration')
    parser.add_argument('--skip_threshold', type=float, default='1e6',
                        help='skipping batch that has large error')

    # Log specifications
    parser.add_argument('--save', type=str, default='RCAN',
                        help='file name to save')
    parser.add_argument('--load', type=str, default='.',
                        help='file name to load')
    parser.add_argument('--resume', type=int, default=0,
                        help='resume from specific checkpoint')
    parser.add_argument('--print_model', action='store_true',
                        help='print model')
    parser.add_argument('--save_models', action='store_true',
                        help='save all intermediate models')
    parser.add_argument('--print_every', type=int, default=100,
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save_results', action='store_true',
                        help='save output results')

    # New options
    parser.add_argument('--n_resgroups', type=int, default=10,
                        help='number of residual groups')
    parser.add_argument('--reduction', type=int, default=16,
                        help='number of feature maps reduction')
    parser.add_argument('--testpath', type=str, default='../test/DIV2K_val_LR_our',
                        help='dataset directory for testing')
    parser.add_argument('--testset', type=str, default='Set5',
                        help='dataset name for testing')
    parser.add_argument('--degradation', type=str, default='BI',
                        help='degradation model: BI, BD')
    # args = []
    # args = parser.parse_known_args()[0]
    # import platform 
    # sys = platform.system()
    # if sys == "Windows":
    #     args = parser.parse_args(args=[])
    # else:
    #     args = parser.parse_args()

    # args = parser.parse_args()


    #     template.set_template(args)

    args = parser.parse_args(args=[])

    args.scale = list(map(lambda x: int(x), args.scale.split('+')))

    if args.epochs == 0:
        args.epochs = 1e8

    for arg in vars(args):
        if vars(args)[arg] == 'True':
            vars(args)[arg] = True
        elif vars(args)[arg] == 'False':
            vars(args)[arg] = False
    return args




args=set_args()

args.file_ACCESS_dir="../data/"
args.file_BARRA_dir="../data/barra_aus/"
args.file_DEM_dir="../DEM/"

init_date=date(1970, 1, 1)
start_date=date(1990, 1, 2)
end_date=date(1990,12,10)
args.channels=0
args.dem=0
args.psl=0
args.zg=1
args.tasmax=1
args.tasmin=0



if args.pr:
    args.channels+=1
if args.zg:
    args.channels+=1
if args.psl:
    args.channels+=1
if args.tasmax:
    args.channels+=1
if args.tasmin:
    args.channels+=1
if args.dem:
    args.channels+=1
access_rgb_mean= 2.9067910245780248e-05*86400

leading_time=217
args.leading_time_we_use=7
args.ensemble=2


In [5]:
import os
import data_processing_tool as dpt
from datetime import timedelta, date, datetime
# import args_parameter as args
import torch,torchvision
import numpy as np

from torch.utils.data import Dataset,random_split
from torchvision import datasets, models, transforms

import time
import random



class ACCESS_BARRA_v2_0(Dataset):
    '''
    scale is size(hr)=size(lr)*scale
    version_3_documention: compare with ver1, I modify:
    1. access file is created on getitem,the file list is access_date,barra,barra_date,time_leading
      in order to read more data like zg etc. more easier, we change access_filepath to access_date

    2. in ver., norm the every inputs 
   
    '''
    def __init__(self,start_date=date(1990, 1, 1),end_date=date(1990,12 , 31),regin="AUS",transform=None,shuffle=True,args=None):
        print("=> BARRA_R & ACCESS_S1 loading")
        print("=> from "+start_date.strftime("%Y/%m/%d")+" to "+end_date.strftime("%Y/%m/%d")+"")
        self.file_BARRA_dir = args.file_BARRA_dir
        self.file_ACCESS_dir = args.file_ACCESS_dir
        self.args=args
        
        self.transform = transform
        self.start_date = start_date
        self.end_date = end_date
        
        self.scale = args.scale[0]
        self.regin = regin
        self.leading_time=217
        self.leading_time_we_use=args.leading_time_we_use

        self.ensemble_access=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']
        self.ensemble=[]
        for i in range(args.ensemble):
            self.ensemble.append(self.ensemble_access[i])
                
        self.dates = self.date_range(start_date, end_date)
        
        
        self.filename_list=self.get_filename_with_time_order(args.file_ACCESS_dir+"pr/daily/")
        if not os.path.exists(args.file_ACCESS_dir+"pr/daily/"):
            print(args.file_ACCESS_dir+"pr/daily/")
            print("no file or no permission")
        
        
        _,_,date_for_BARRA,time_leading=self.filename_list[0]
        if shuffle:
            random.shuffle(self.filename_list)
        
        
        if not os.path.exists("/g/data/ma05/BARRA_R/v1/forecast/spec/accum_prcp/1990/01/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19900109T0600Z.sub.nc"):
            print(self.file_BARRA_dir)
            print("no file or no permission!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        data_high=dpt.read_barra_data_fc_get_lat_lon(self.file_BARRA_dir,date_for_BARRA)
        self.lat=data_high[1]
        self.lon=data_high[1]
        self.shape=(79,94)
        if self.args.dem:
            data_dem=dpt.add_lat_lon( dpt.read_dem(args.file_DEM_dir+"dem-9s1.tif"))
            self.dem_data=dpt.interp_tensor_2d(dpt.map_aust_old(data_dem,xrarray=False) ,self.shape )
        

        
    def __len__(self):
        return len(self.filename_list)
    

    def date_range(self,start_date, end_date):
        """This function takes a start date and an end date as datetime date objects.
        It returns a list of dates for each date in order starting at the first date and ending with the last date"""
        return [start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]

    
    def get_filename_with_no_time_order(self,rootdir):
        '''get filename first and generate label '''
        _files = []
        list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
        for i in range(0,len(list)):
            path = os.path.join(rootdir,list[i])
            if os.path.isdir(path):
                _files.extend(self.get_filename_with_no_time_order(path))
            if os.path.isfile(path):
                if path[-3:]==".nc":
                    _files.append(path)
        return _files
    
    def get_filename_with_time_order(self,rootdir):
        '''get filename first and generate label ,one different w'''
        _files = []
        for en in self.ensemble:
            for date in self.dates:

#                 filename="da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"cd
                access_path=rootdir+en+"/"+"da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"
#                 print(access_path)
                if os.path.exists(access_path):
                    for i in range(self.leading_time_we_use):
                        if date==self.end_date and i==1:
                            break
                        path=[]
                        path.append(en)
                        barra_date=date+timedelta(i)
                        path.append(date)
                        path.append(barra_date)
                        path.append(i)
                        _files.append(path)
    
    #最后去掉第一行，然后shuffle
        return _files

    

        
    def __getitem__(self, idx):
        '''
        from filename idx get id
        return lr,hr
        '''
        t=time.time()
        
        #read_data filemame[idx]
        en,access_date,barra_date,time_leading=self.filename_list[idx]
        

        lr=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"pr")
        lr=np.expand_dims(lr,axis=2)
        label=dpt.read_barra_data_fc(self.file_BARRA_dir,barra_date)

        if self.args.zg:
            lr_zg=np.expand_dims(dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"zg"),axis=2)
            lr=np.concatenate((lr,lr_zg),axis=2)

        if self.args.psl:
            lr_psl=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"psl")

        if self.args.tasmax:
            lr_tasmax=np.expand_dims(dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"tasmax"),axis=2)
            lr=np.concatenate((lr,lr_tasmax),axis=2)

        if self.args.tasmin:
            lr_tasmin=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"tasmin")
            
            
            
        if self.transform:#channel 数量需要整理！！
            return self.transform(lr),self.transform(label),torch.tensor(int(barra_date.strftime("%Y%m%d"))),torch.tensor(time_leading)
#             if self.args.channels==6:
#                 return self.transform(lr),self.transform(self.dem_data),self.transform(lr_psl),self.transform(lr_zg),self.transform(lr_tasmax),self.transform(lr_tasmin),self.transform(label),torch.tensor(int(barra_date.strftime("%Y%m%d"))),torch.tensor(time_leading)
#             elif self.args.channels==3:
#                 return self.transform(lr),self.transform(lr_zg),self.transform(lr_tasmax),self.transform(label),torch.tensor(int(barra_date.strftime("%Y%m%d"))),torch.tensor(time_leading)
        else:
            return lr*86400,label,torch.tensor(int(date_for_BARRA.strftime("%Y%m%d"))),torch.tensor(time_leading)

#         if self.transform:#channel 数量需要整理！！
#             if self.args.channels==6:
#                 return self.transform(lr),self.transform(self.dem_data),self.transform(lr_psl),self.transform(lr_zg),self.transform(lr_tasmax),self.transform(lr_tasmin),self.transform(label),torch.tensor(int(date_for_BARRA.strftime("%Y%m%d"))),torch.tensor(time_leading)
#             elif self.args.channels==2:
#                 return self.transform(lr*86400),self.transform(self.dem_data),self.transform(label),torch.tensor(int(date_for_BARRA.strftime("%Y%m%d"))),torch.tensor(time_leading)
# #             e self.args.channels==2:
#         else:
#             return lr*86400,label,torch.tensor(int(date_for_BARRA.strftime("%Y%m%d"))),torch.tensor(time_leading)



In [6]:
train_transforms = transforms.Compose([
    transforms.ToTensor()
])
from PrepareData import ACCESS_BARRA_v2_0

data_set=ACCESS_BARRA_v2_0(
    start_date=date(1990, 1, 2),
    end_date=date(2011, 12, 28),
    transform=train_transforms,
    args=args,
)


=> BARRA_R & ACCESS_S1 loading
=> from 1990/01/02 to 2012/12/28
../data/barra_aus/
no file or no permission!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [7]:
len(data_set)

15442

In [ ]:
train_data,val_data=random_split(data_set,[int(len(data_set)*0.8),len(data_set)-int(len(data_set)*0.8)])

In [ ]:
args.n_threads=0
train_dataloders =DataLoader(train_data,
                                        batch_size=args.batch_size,
                                        shuffle=True,
                            num_workers=args.n_threads)
val_dataloders =DataLoader(val_data,
                                        batch_size=args.batch_size,
                                        shuffle=True,
                          num_workers=args.n_threads)

In [ ]:
for batch, (lr,hr,_,_) in enumerate(train_dataloders):
    print(batch)
    print(torch.max(lr)    )

In [ ]:
lr[0][0][0]

In [ ]:
a=93452348572695624783562956
b=236498726391862948612783461987346
a+b
